In [1]:
from rdkit import *
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
import graphviz
import joblib
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
from math import *
import sklearn
from sklearn.decomposition import PCA
from sklearn import ensemble
from sklearn import tree
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve,auc,roc_auc_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import export_graphviz
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv('TLR4_activity_SMILES.csv',usecols=[1],names=None)

df_li=df.values.tolist()
result=[]
for s_li in df_li:
    result.append(s_li[0])

mol=[Chem.MolFromSmiles(i) for i in result]
df['mol']=mol
df['mol'] = df['mol'].map(lambda x: False if x == None else True)
del_index = df[df['mol'] == False].index
df2 = df.drop(del_index)
mols = [Chem.MolFromSmiles(smile) for smile in df2['SMILES']]
maccskeys = []
for m in mols:
    maccskey = [x for x in AllChem.GetMACCSKeysFingerprint(m)]
    maccskeys.append(maccskey)
maccskeys = np.array(maccskeys)

df=pd.read_csv('TLR4_activity_SMILES.csv',names=None)

merge=pd.concat([df2,df],axis=1,join='inner')
merge.activity=merge.activity.astype(str).map({'Inactive':0, 'Active':1})

y=merge.activity
x=pd.DataFrame(maccskeys)
m=pd.DataFrame(x)
X=maccskeys

scaler = MinMaxScaler()
X = scaler.fit_transform(x)
#joblib.dump(X,'xydataset/geneid_x_s.pkl')
#joblib.dump(y,'xydataset/geneid_y_s.pkl')



In [3]:
#SMOTE算法及其python实现
from sklearn.neighbors import NearestNeighbors

class Smote:
    def __init__(self, samples, N=100, k=5):
        self.n_samples, self.n_attrs = samples.shape
        self.N = N
        self.k = k
        self.samples = samples
        self.newindex = 0
        self.synthetic = np.zeros((self.n_samples * int(N / 100), self.n_attrs))

    def over_sampling(self):
        neighbors = NearestNeighbors(n_neighbors=self.k).fit(self.samples)

        for i in range(len(self.samples)):
            nnarray = neighbors.kneighbors(self.samples[i].reshape(1,-1), return_distance=False)[0]
            self._populate(int(self.N/100), i, nnarray)

        return self.synthetic

    def _populate(self, N, i, nnarray):
        for j in range(N):
            nn = random.randint(0, len(nnarray)-1)
            dif = self.samples[nnarray[nn]] - self.samples[i]
            gap = random.random()
            self.synthetic[self.newindex] = self.samples[i] + gap * dif
            self.newindex += 1
            
X=maccskeys
s=Smote(X,N=100)
x=s.over_sampling()

In [4]:
#knn
knn = KNeighborsClassifier()

# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
 
# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []
 
# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]
 
    # 训练模型
    knn.fit(X_train, y_train)
 
 # 验证并评估模型在验证集上的性能
    y_validation_pred = knn.predict(X_validation)
    y_validation_pred_prob = knn.predict_proba(X_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))
 
# 输出每个模型在不同指标下的性能表现
print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/knn/scorelist/acc/geneid_knn_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/knn/scorelist/f1/geneid_knn_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/knn/scorelist/auc/geneid_knn_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/knn/scorelist/recall/geneid_knn_recall.pkl')
#joblib.dump(precision_score,'model_pkl/knn/scorelist/precision/geneid_knn_precision.pkl')
#joblib.dump(knn,'model_pkl/knn/geneid_s_knn.pkl')

0.7993851911253675
0.7459518990103772
0.8802438568964215
0.7261117911056711
0.7786269420634401


In [5]:
#svm
svm = SVC(probability=True)

# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
 
# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []
 
# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]
 
    # 训练模型
    svm.fit(X_train, y_train)
 
 # 验证并评估模型在验证集上的性能
    y_validation_pred = svm.predict(X_validation)
    y_validation_pred_prob = svm.predict_proba(X_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))
 
# 输出每个模型在不同指标下的性能表现
print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/svm/scorelist/acc/geneid_svm_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/svm/scorelist/f1/geneid_svm_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/svm/scorelist/auc/geneid_svm_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/svm/scorelist/recall/geneid_svm_recall.pkl')
#joblib.dump(precision_score,'model_pkl/svm/scorelist/precision/geneid_svm_precision.pkl')
#joblib.dump(svm,'model_pkl/svm/geneid_s_svm.pkl')



0.8293504410585406
0.7827217152843057
0.8892568339689385
0.7580753015001471
0.8206151351901532


In [6]:
#rfc

# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []

# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]

    # 定义参数搜索空间，并进行网格搜索
    param_grid = {'n_estimators': [10, 50, 100], 
                  'max_depth': [3, 5, 7]}
    rfc = RandomForestClassifier()
    gs = GridSearchCV(rfc, param_grid, cv=5)
    gs.fit(X_train, y_train)  # 搜索最佳超参数组合（很耗时！）
    # 携带最佳超参数组合的RFC对象
    best_rfc = gs.best_estimator_

    # 训练模型，并在验证集上评估性能
    best_rfc.fit(X_train, y_train)
    y_validation_pred = best_rfc.predict(X_validation)
    y_validation_pred_prob = best_rfc.predict_proba(X_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))


 
# 输出每个模型在不同指标下的性能表现
print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/rfc/scorelist/acc/geneid_rfc_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/rfc/scorelist/f1/geneid_rfc_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/rfc/scorelist/auc/geneid_rfc_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/rfc/scorelist/recall/geneid_rfc_recall.pkl')
#joblib.dump(precision_score,'model_pkl/rfc/scorelist/precision/geneid_rfc_precision.pkl')
#joblib.dump(best_rfc,'model_pkl/rfc/geneid_s_rfc.pkl')



0.8201817695803261
0.7702578607014174
0.894582536256068
0.7495195195195195
0.7998920369037492


In [7]:
#dt

# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []

# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]

    # 定义参数搜索空间，并进行网格搜索
    param_grid = {
    'max_depth': [5, 10, 15],
    'min_samples_leaf': [1, 2, 3],
    'criterion': ['gini', 'entropy']
    }
    dt = DecisionTreeClassifier()
    gs = GridSearchCV(dt,param_grid,cv=5)
    gs.fit(X_train, y_train)  # 搜索最佳超参数组合（很耗时！）
    # 携带最佳超参数组合的RFC对象
    best_dt = gs.best_estimator_

    # 训练模型，并在验证集上评估性能
    best_dt.fit(X_train, y_train)
    y_validation_pred = best_dt.predict(X_validation)
    y_validation_pred_prob = best_dt.predict_proba(X_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))


 
# 输出每个模型在不同指标下的性能表现
print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/dt/scorelist/acc/geneid_dt_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/dt/scorelist/f1/geneid_dt_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/dt/scorelist/auc/geneid_dt_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/dt/scorelist/recall/geneid_dt_recall.pkl')
#joblib.dump(precision_score,'model_pkl/dt/scorelist/precision/geneid_dt_precision.pkl')
#joblib.dump(best_dt,'model_pkl/dt/geneid_s_dt.pkl')



0.7993584603047313
0.7324964593130379
0.8300852554993975
0.7428545963840081
0.7288523195707649


In [8]:
#mnb
mnb = MultinomialNB()

# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
 
# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []
 
# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]
 
    # 训练模型
    mnb.fit(X_train, y_train)
 
 # 验证并评估模型在验证集上的性能
    y_validation_pred = mnb.predict(X_validation)
    y_validation_pred_prob = mnb.predict_proba(X_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))
 
# 输出每个模型在不同指标下的性能表现
print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/nb/scorelist/acc/ABL1_nb_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/nb/scorelist/f1/ABL1_nb_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/nb/scorelist/auc/ABL1_nb_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/nb/scorelist/recall/ABL1_nb_recall.pkl')
#joblib.dump(precision_score,'model_pkl/nb/scorelist/precision/ABL1_nb_precision.pkl')
#joblib.dump(svm,'model_pkl/nb/ABL1_s_nb.pkl')



0.6910184442662389
0.6255703144218765
0.8209046640628144
0.5798886858325983
0.6860923902375188


In [9]:
#adaboost

# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []

# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    x_train, x_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]

    # 定义参数搜索空间，并进行网格搜索
    param_grid = {
        'n_estimators': [10, 50, 100, 200, 500], 
        'learning_rate': [0.01, 0.1, 1, 10]
    }
    adaboost = AdaBoostClassifier(base_estimator=best_dt)
    gs = GridSearchCV(adaboost,param_grid,cv=5)
    gs.fit(x_train,y_train)  # 搜索最佳超参数组合（很耗时！）
    # 携带最佳超参数组合的AdaBoost对象
    best_adaboost = gs.best_estimator_

    # 使用最佳超参数组合的分类器进行拟合训练
    best_adaboost.fit(x_train, y_train)

    # 验证并评估模型在验证集上的性能
    y_validation_pred = best_adaboost.predict(x_validation)
    y_validation_pred_prob = best_adaboost.predict_proba(x_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))
 
# 输出每个模型在不同指标下的性能表现

print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/adaboost/scorelist/acc/ABL1_adaboost_lda_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/adaboost/scorelist/f1/ABL1_adaboost_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/adaboost/scorelist/auc/ABL1_adaboost_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/adaboost/scorelist/recall/ABL1_adaboost_recall.pkl')
#joblib.dump(precision_score,'model_pkl/adaboost/scorelist/precision/ABL1_adaboost_precision.pkl')
#joblib.dump(svm,'model_pkl/adaboost/ABL1_s_adaboost.pkl')



0.7787757284148624
0.7116674629718108
0.8833101741766403
0.6993403813138158
0.7254494827222695


In [10]:
# 定义LDA模型
lda = LinearDiscriminantAnalysis()

# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
 
# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []
 
# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]
 
    # 训练模型
    lda.fit(X_train, y_train)
 
 # 验证并评估模型在验证集上的性能
    y_validation_pred = lda.predict(X_validation)
    y_validation_pred_prob = lda.predict_proba(X_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))
 
# 输出每个模型在不同指标下的性能表现
print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/LDA/scorelist/acc/ABL1_lda_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/LDA/scorelist/f1/ABL1_lda_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/LDA/scorelist/auc/ABL1_lda_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/LDA/scorelist/recall/ABL1_lda_recall.pkl')
#joblib.dump(precision_score,'model_pkl/LDA/scorelist/precision/ABL1_lda_precision.pkl')
#joblib.dump(svm,'model_pkl/LDA/ABL1_s_lda.pkl')



0.7648489708634055
0.6959248573187573
0.8185722231866889
0.6869154310484786
0.7123037943393894


In [11]:
#lr

# 定义逻辑回归模型
lr = LogisticRegression()

# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
 
# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []
 
# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]
 
    # 训练模型
    lr.fit(X_train, y_train)
 
 # 验证并评估模型在验证集上的性能
    y_validation_pred = lr.predict(X_validation)
    y_validation_pred_prob = lr.predict_proba(X_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))
 
# 输出每个模型在不同指标下的性能表现
print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/lr/scorelist/acc/ABL1_lr_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/lr/scorelist/f1/ABL1_lr_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/lr/scorelist/auc/ABL1_lr_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/lr/scorelist/recall/ABL1_lr_recall.pkl')
#joblib.dump(precision_score,'model_pkl/lr/scorelist/precision/ABL1_lr_precision.pkl')
#joblib.dump(svm,'model_pkl/lr/ABL1_s_lr.pkl')



/home/qinyf/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/qinyf/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

0.8133386794974605
0.7545797553994276
0.8851987757842149
0.7568445052439581
0.7623208794084931


In [12]:
#ensemble

# 创建各个基分类器
nn_clf = MLPClassifier()

# 创建AdaBoost分类器
voting_clf = VotingClassifier(
    estimators=[
        ('knn', knn), 
        ('svm', svm), 
        ('rf', best_rfc), 
        ('dt', best_dt), 
        ('nb', mnb), 
        ('adaBoost', best_adaboost), 
        ('lda', lda), 
        ('lr', lr), 
        ('mlp', nn_clf)],
    voting='soft',
    n_jobs=-1
)


# 定义五折交叉验证迭代器
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
 
# 定义一个空列表，用于保存每个模型在验证集上的准确率
accuracy_scores, auc_scores, f1_scores, recall_scores, precision_scores = [], [], [], [], []
 
# 开始五折交叉验证
for train_index, validation_index in kfold.split(X):
    # 将数据集分成训练集和验证集
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]
 
    # 训练模型
    voting_clf.fit(X_train, y_train)
 
 # 验证并评估模型在验证集上的性能
    y_validation_pred = voting_clf.predict(X_validation)
    y_validation_pred_prob = voting_clf.predict_proba(X_validation)[:, 1] # 取正例的概率值，用于计算AUC
    accuracy_scores.append(accuracy_score(y_validation, y_validation_pred))
    auc_scores.append(roc_auc_score(y_validation, y_validation_pred_prob))
    f1_scores.append(f1_score(y_validation, y_validation_pred))
    recall_scores.append(recall_score(y_validation, y_validation_pred))
    precision_scores.append(precision_score(y_validation, y_validation_pred))
 
# 输出每个模型在不同指标下的性能表现
print(np.mean(accuracy_scores))
print(np.mean(f1_scores))
print(np.mean(auc_scores))
print(np.mean(precision_scores))
print(np.mean(recall_scores))
#joblib.dump(accuracy_scores,'model_pkl/ensemble_scorelist/acc/ABL1_acc.pkl')
#joblib.dump(f1_scores,'model_pkl/ensemble_scorelist/f1/ABL1_f1.pkl')
#joblib.dump(auc_scores,'model_pkl/ensemble_scorelist/auc/ABL1_auc.pkl')
#joblib.dump(recall_scores,'model_pkl/ensemble_scorelist/recall/ABL1_recall.pkl')
#joblib.dump(precision_score,'model_pkl/ensemble_scorelist/precision/ABL1_precision.pkl')
#joblib.dump(voting_clf,'model_pkl/ABL1_s_ensemble.pkl')



0.8017375033413525
0.7469041454326264
0.8929774660054047
0.7262566844919787
0.7785088006281489


In [13]:
mol=Chem.MolFromSmiles('OC1=CC=C(CC2=CC=C(O)C=C2)C=C1')#BPF
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


In [14]:
mol=Chem.MolFromSmiles('C1=CC=C2C(=C1)C3=CC=CC=C3C2(C4=CC5=C(C=C4)C(=O)OC5=O)C6=CC7=C(C=C6)C(=O)OC7=O')#BPAF
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[1]


In [15]:
mol=Chem.MolFromSmiles('OC1=CC=C(C=C1)S(=O)(=O)C1=CC=C(O)C=C1')#BPS
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


In [16]:
mol=Chem.MolFromSmiles('CC(C1=CC=CC=C1)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1')#BPAP
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[0]
[1]
[1]
[1]
[0]
[1]
[1]


In [17]:
mol=Chem.MolFromSmiles('Cl.CC1=NN(C(=N)N=C(N)N)C(C)(C)C1')#BPC
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]


In [18]:
mol=Chem.MolFromSmiles('CC(CCC(O)=O)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1')#BPACID
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[0]
[1]
[0]
[0]
[1]
[1]
[1]
[1]
[1]


In [19]:
mol=Chem.MolFromSmiles('OC1=CC=C(C=C1)C1(CCCCC1)C1=CC=C(O)C=C1')#BPZ
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[1]
[0]
[1]
[1]


In [20]:
mol=Chem.MolFromSmiles('CC(C)(C1=CC(=C(O)C=C1)C1=CC=CC=C1)C1=CC(=C(O)C=C1)C1=CC=CC=C1')#BPOPPA
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[0]
[1]
[1]
[1]
[0]
[1]
[1]


In [21]:
mol=Chem.MolFromSmiles('CCC(C)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1')#BPB
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[0]
[1]
[1]
[1]
[0]
[1]
[1]


In [22]:
mol=Chem.MolFromSmiles('OC1=CC=C(C=C1)C1(C2=C(C=CC=C2)C2=C1C=CC=C2)C1=CC=C(O)C=C1')#BPFL
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[0]
[0]
[1]
[1]


In [23]:
mol=Chem.MolFromSmiles('CC(C1=CC=C(OC#N)C=C1)C1=CC=C(OC#N)C=C1')#1,-is(4-yanatophenyl)thane
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[0]
[0]
[1]
[1]


In [24]:
mol=Chem.MolFromSmiles('OC(=O)COC1=CC=C(Cl)C=C1')#4-CP
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[0]
[1]
[1]
[1]
[0]
[0]
[1]


In [25]:
mol=Chem.MolFromSmiles('CC(=C)C(=O)OCC(O)COC1=CC=C(C=C1)C(C)(C)C1=CC=C(OCC(O)COC(=O)C(C)=C)C=C1')#BPAGM
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[0]
[0]
[0]
[1]
[1]
[1]
[0]
[1]
[1]


In [26]:
mol=Chem.MolFromSmiles('CC(C)(C1=CC=C(OC#N)C=C1)C1=CC=C(OC#N)C=C1')#1,3-Bis(4-cyanophenyl)propane
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[1]
[0]
[1]
[1]


In [27]:
mol=Chem.MolFromSmiles('O=C(N1CCCCC1)C1=CC2=C(OCCO2)C=C1')#BDP
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[0]
[1]
[0]
[0]
[1]
[1]
[1]
[1]
[1]


In [28]:
mol=Chem.MolFromSmiles('CC(C)(C1=CC=C(OC2=CC3=C(C=C2)C(=O)OC3=O)C=C1)C1=CC=C(OC2=CC3=C(C=C2)C(=O)OC3=O)C=C1')#BPADA
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[1]


In [29]:
mol=Chem.MolFromSmiles('CC(=O)OC1=CC=C(C=C1)C(C)(C)C1=CC=C(OC(C)=O)C=C1')#BPAD
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


In [30]:
mol=Chem.MolFromSmiles('CC(C)(C1=CC=C(OCC=C)C=C1)C1=CC=C(OCC=C)C=C1')#BPE
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[0]
[1]
[1]
[1]
[0]
[1]
[1]


In [31]:
mol=Chem.MolFromSmiles('CC(C)(C1=CC=C(OC#N)C=C1)C1=CC=C(OC#N)C=C1')#BPACE
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[1]
[1]
[1]
[1]
[1]
[0]
[1]
[1]


In [32]:
mol=Chem.MolFromSmiles('CC(C)(C1=CC=C(OCCO)C=C1)C1=CC=C(OCCO)C=C1')#Diano 22
maccskey=AllChem.GetMACCSKeysFingerprint(mol)
maccs= np.array(maccskey).reshape(1,-1)
print(knn.predict(maccs))
print(svm.predict(maccs))
print(best_rfc.predict(maccs))
print(best_dt.predict(maccs))
print(mnb.predict(maccs))
print(best_adaboost.predict(maccs))
print(lda.predict(maccs))
print(lr.predict(maccs))
print(voting_clf.predict(maccs))


[1]
[0]
[0]
[1]
[1]
[1]
[0]
[1]
[1]
